In [1]:
import pandas as pd

# Đọc tập tin CSV vào DataFrame
file_path = 'Data/cheese_data.csv'
data = pd.read_csv(file_path)

# Hiển thị một vài dòng đầu của dữ liệu
data.head()

,CheeseId,ManufacturerProvCode,ManufacturingTypeEn,MoisturePercent,FlavourEn,CharacteristicsEn,Organic,CategoryTypeEn,MilkTypeEn,MilkTreatmentTypeEn,RindTypeEn,CheeseName,FatLevel
0,228,NB,Farmstead,47.0,"Sharp, lactic",Uncooked,0,Firm Cheese,Ewe,Raw Milk,Washed Rind,Sieur de Duplessis (Le),lower fat
1,242,NB,Farmstead,47.9,"Sharp, lactic, lightly caramelized",Uncooked,0,Semi-soft Cheese,Cow,Raw Milk,Washed Rind,Tomme Le Champ Doré,lower fat
2,301,ON,Industrial,54.0,"Mild, tangy, and fruity","Pressed and cooked cheese, pasta filata, inter...",0,Firm Cheese,Cow,Pasteurized,NaN,Provolone Sette Fette (Tre-Stelle),lower fat
3,303,NB,Farmstead,47.0,Sharp with fruity notes and a hint of wild honey,NaN,0,Veined Cheeses,Cow,Raw Milk,NaN,Geai Bleu (Le),lower fat
4,319,NB,Farmstead,49.4,Softer taste,NaN,1,Semi-soft Cheese,Cow,Raw Milk,Washed Rind,Gamin (Le),lower fat


In [3]:
# Kiểm tra kích thước của dữ liệu
print("Kích thước dữ liệu:", data.shape)

# Kiểm tra thông tin tổng quan của dữ liệu
data.info()

# Thống kê mô tả cơ bản
data.describe()

Kích thước dữ liệu: (1042, 13)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CheeseId              1042 non-null   int64  
 1   ManufacturerProvCode  1042 non-null   object 
 2   ManufacturingTypeEn   1042 non-null   object 
 3   MoisturePercent       1028 non-null   float64
 4   FlavourEn             801 non-null    object 
 5   CharacteristicsEn     643 non-null    object 
 6   Organic               1042 non-null   int64  
 7   CategoryTypeEn        1019 non-null   object 
 8   MilkTypeEn            1041 non-null   object 
 9   MilkTreatmentTypeEn   977 non-null    object 
 10  RindTypeEn            721 non-null    object 
 11  CheeseName            1042 non-null   object 
 12  FatLevel              1042 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 106.0+ KB


,CheeseId,MoisturePercent,Organic
count,1042.000000,1028.000000,1042.000000
mean,1560.633397,47.069747,0.095010
std,451.811164,9.592647,0.293369
min,228.000000,12.000000,0.000000
25%,1280.250000,40.000000,0.000000
50%,1548.500000,46.000000,0.000000
75%,1901.750000,52.000000,0.000000
max,2391.000000,92.000000,1.000000


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

# Xử lý giá trị thiếu trong cột 'FlavourEn' bằng cách thay thế bằng giá trị phổ biến nhất
data['FlavourEn'] = data['FlavourEn'].fillna(data['FlavourEn'].mode()[0])

# Loại bỏ các hàng có giá trị thiếu trong các cột mục tiêu hoặc đặc trưng chính
data.dropna(subset=['CategoryTypeEn', 'MoisturePercent', 'CheeseName'], inplace=True)

# Mã hóa cột CategoryTypeEn và CheeseName (vì đây là bài toán phân loại)
label_encoder_category = LabelEncoder()
data['CategoryTypeEn'] = label_encoder_category.fit_transform(data['CategoryTypeEn'])

label_encoder_cheese_name = LabelEncoder()
data['CheeseName'] = label_encoder_cheese_name.fit_transform(data['CheeseName'])

# Áp dụng TF-IDF để chuyển đổi cột văn bản 'FlavourEn' thành các đặc trưng số
tfidf_vectorizer = TfidfVectorizer(max_features=100)  # Giới hạn số lượng đặc trưng để giảm độ phức tạp
flavour_tfidf = tfidf_vectorizer.fit_transform(data['FlavourEn']).toarray()

# Kết hợp các đặc trưng TF-IDF vào DataFrame
flavour_df = pd.DataFrame(flavour_tfidf, columns=tfidf_vectorizer.get_feature_names_out())
data = pd.concat([data.reset_index(drop=True), flavour_df], axis=1)

# Loại bỏ các cột không phải số mà không cần thiết sau khi chuyển đổi
data.drop(['FlavourEn', 'ManufacturerProvCode', 'ManufacturingTypeEn', 'CharacteristicsEn', 
           'MilkTypeEn', 'MilkTreatmentTypeEn', 'RindTypeEn', 'FatLevel'], axis=1, inplace=True)

# Kiểm tra lại kiểu dữ liệu để đảm bảo tất cả đặc trưng đều là số
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Columns: 105 entries, CheeseId to with
dtypes: float64(101), int32(2), int64(2)
memory usage: 816.7 KB


In [19]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Đọc dữ liệu từ file CSV
file_path = 'Data/cheese_data.csv'
data = pd.read_csv(file_path)

# Xử lý giá trị thiếu trong cột 'FatLevel' và 'CategoryTypeEn'
data.dropna(subset=['FatLevel', 'CategoryTypeEn'], inplace=True)

# Mã hóa cột FatLevel và CategoryTypeEn
label_encoder_fat = LabelEncoder()
data['FatLevel'] = label_encoder_fat.fit_transform(data['FatLevel'])

label_encoder_category = LabelEncoder()
data['CategoryTypeEn'] = label_encoder_category.fit_transform(data['CategoryTypeEn'])

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(data[['FatLevel']], data['CategoryTypeEn'], test_size=0.2, random_state=42)

# Huấn luyện mô hình Random Forest
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print("Độ chính xác phân loại CategoryTypeEn dựa trên FatLevel:", accuracy)

# Giải mã nhãn thực tế và nhãn dự đoán về tên loại phô mai
y_test_labels = label_encoder_category.inverse_transform(y_test)
y_pred_labels = label_encoder_category.inverse_transform(y_pred)

# In ra các giá trị dự đoán và thực tế dưới dạng tên loại phô mai
result_df = pd.DataFrame({'Giá trị thực tế (Loại phô mai)': y_test_labels, 'Giá trị dự đoán (Loại phô mai)': y_pred_labels})
print("\nKết quả phân loại CategoryTypeEn dựa trên FatLevel:")
print(result_df)


Độ chính xác phân loại CategoryTypeEn dựa trên FatLevel: 0.4117647058823529

Kết quả phân loại CategoryTypeEn dựa trên FatLevel:
    Giá trị thực tế (Loại phô mai) Giá trị dự đoán (Loại phô mai)
0                      Firm Cheese                    Soft Cheese
1                      Soft Cheese                    Soft Cheese
2                      Soft Cheese                    Soft Cheese
3                     Fresh Cheese                    Soft Cheese
4                 Semi-soft Cheese                    Soft Cheese
..                             ...                            ...
199               Semi-soft Cheese                    Soft Cheese
200                    Soft Cheese                    Soft Cheese
201                    Firm Cheese                    Firm Cheese
202                   Fresh Cheese                    Soft Cheese
203                    Soft Cheese                    Soft Cheese

[204 rows x 2 columns]


In [23]:
# Import các thư viện cần thiết
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Đọc dữ liệu từ file CSV
file_path = 'Data/cheese_data.csv'
data = pd.read_csv(file_path)

# Xử lý giá trị thiếu trong cột 'MoisturePercent' và 'FatLevel'
data.dropna(subset=['MoisturePercent', 'FatLevel'], inplace=True)

# Mã hóa cột FatLevel (nếu là kiểu chuỗi)
if data['FatLevel'].dtype == 'object':
    label_encoder = LabelEncoder()
    data['FatLevel'] = label_encoder.fit_transform(data['FatLevel'])

# Loại bỏ các cột không phải số (trừ cột 'MoisturePercent')
non_numeric_columns = data.select_dtypes(include=['object']).columns
X_features = data.drop(columns=non_numeric_columns.tolist() + ['MoisturePercent'])

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
y_moisture = data['MoisturePercent']
X_train, X_test, y_train, y_test = train_test_split(X_features, y_moisture, test_size=0.2, random_state=42)

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Khởi tạo và huấn luyện mô hình Random Forest Regressor
rf_regressor = RandomForestRegressor(random_state=42)
rf_regressor.fit(X_train, y_train)

# Dự đoán độ ẩm của phô mai trên tập kiểm tra
y_pred = rf_regressor.predict(X_test)

# Tạo DataFrame để hiển thị kết quả thực tế và dự đoán độ ẩm
predicted_data = pd.DataFrame({
    'Độ ẩm thực tế': y_test,
    'Độ ẩm dự đoán': y_pred
})

# Hiển thị 10 kết quả dự đoán đầu tiên
print(predicted_data.head(10))

# Đánh giá mô hình
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Hiển thị kết quả đánh giá mô hình
print(f"\nMean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")


     Độ ẩm thực tế  Độ ẩm dự đoán
436           50.0          52.32
543           52.0          57.42
396           26.0          39.48
112           41.0          38.36
431           50.0          49.92
428           46.0          49.61
166           35.0          37.65
245           50.0          53.19
673           45.0          42.06
553           60.0          52.51

Mean Absolute Error (MAE): 6.46
Mean Squared Error (MSE): 92.02
Root Mean Squared Error (RMSE): 9.59


In [97]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Bước 1: Tải dữ liệu
data = pd.read_csv('Data/cheese_data.csv')

# Kiểm tra thông tin dữ liệu
print("Thông tin dữ liệu:\n", data.info())

# Kiểm tra xem có giá trị thiếu trong dữ liệu không
print("Kiểm tra giá trị thiếu:\n", data.isnull().sum())

# Kiểm tra phân bố lớp của nhãn
print("Phân bố lớp của nhãn:\n", data['CheeseName'].value_counts())

# Bước 2: Tiền xử lý dữ liệu
X = data[['FlavourEn']]  # Hương vị của phô mai
y = data['CheeseName']  # Tên phô mai

# Chuyển nhãn tên phô mai thành số với LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Sử dụng OneHotEncoder để chuyển đổi hương vị thành đặc trưng số
preprocessor = ColumnTransformer(
    transformers=[
        ('flavour', OneHotEncoder(handle_unknown='ignore'), ['FlavourEn'])
    ])

# Bước 3: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Bước 4: Khởi tạo mô hình Random Forest với pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Huấn luyện mô hình
model.fit(X_train, y_train)

# Bước 5: Dự đoán và đánh giá mô hình
y_pred = model.predict(X_test)

# Đánh giá mô hình
print("Độ chính xác của mô hình:", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại:\n", classification_report(y_test, y_pred, zero_division=1))

# Chuyển đổi nhãn từ dạng số về tên phô mai
y_pred_labels = label_encoder.inverse_transform(y_pred)
y_test_labels = label_encoder.inverse_transform(y_test)

# In ra một số dự đoán và nhãn thực tế
print("Một số dự đoán và nhãn thực tế:")
for i in range(10):
    print(f"Dự đoán: {y_pred_labels[i]} | Thực tế: {y_test_labels[i]}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CheeseId              1042 non-null   int64  
 1   ManufacturerProvCode  1042 non-null   object 
 2   ManufacturingTypeEn   1042 non-null   object 
 3   MoisturePercent       1028 non-null   float64
 4   FlavourEn             801 non-null    object 
 5   CharacteristicsEn     643 non-null    object 
 6   Organic               1042 non-null   int64  
 7   CategoryTypeEn        1019 non-null   object 
 8   MilkTypeEn            1041 non-null   object 
 9   MilkTreatmentTypeEn   977 non-null    object 
 10  RindTypeEn            721 non-null    object 
 11  CheeseName            1042 non-null   object 
 12  FatLevel              1042 non-null   object 
dtypes: float64(1), int64(2), object(10)
memory usage: 106.0+ KB
Thông tin dữ liệu:
 None
Kiểm tra giá trị thiếu:
 CheeseId     